## Analysis

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
import datetime
import seaborn as sns
import matplotlib.pyplot as plt

# Impact Of Airline Tweets in the Sentiment Of A Conversation

Dataset of conversations & Dates as downloaded here
https://drive.google.com/open?id=13v22L7LXIElILm2M4vfdfeo9sMjyqIOM

In [2]:
df = pd.read_csv('rawdates.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df['mean_sentiment'].astype(float, inplace=True).describe()

count    372429.000000
mean          0.542905
std           0.584006
min           0.000000
25%           0.000000
50%           0.500000
75%           1.000000
max           2.000000
Name: mean_sentiment, dtype: float64

In [4]:
df.head()

,Unnamed: 0,False,conversation_id,raw_tweets_info,conversation_length,tweet_ids,user_ids,airlines_involved,mean_sentiment,sentiments,full_text,dates
0,0,0,5,"[(747530773037780992, 31132554), (747526794446...",2,"[747526794446802944, 747530773037780992]","[22873368, 31132554]",['No airlines involved'],NaN,"[nan, 1]",['Hey @British_Airways our @easyJet flight Ven...,"['Mon Jun 27 20:27:30 +0000 2016', 'Mon Jun 27..."
1,1,1,6,"[(747530681509625860, 417945325), (74745123859...",2,"[747451238598475776, 747530681509625860]","[55904765, 417945325]",['No airlines involved'],0.0,"[0, 0]",['@easyJet thanks for looking after my luggage...,"['Mon Jun 27 15:27:16 +0000 2016', 'Mon Jun 27..."
2,2,2,11,"[(747530531475173376, 597673831), (74752831114...",2,"[747528311144734720, 747530531475173376]","[427951650, 597673831]",['No airlines involved'],0.5,"[1, 0]","[""@British_Airways I booked via Expedia. I'm n...","['Mon Jun 27 20:33:32 +0000 2016', 'Mon Jun 27..."
3,3,3,12,"[(747530509098450944, 1624311463), (7475297117...",2,"[747529711782244352, 747530509098450944]","[1624311463, 1624311463]",['No airlines involved'],0.0,"[0, 0]",['@easyJet is there anyway to travel with 2 ca...,"['Mon Jun 27 20:39:06 +0000 2016', 'Mon Jun 27..."
4,4,4,14,"[(747530348561571840, 154605692), (74752081839...",2,"[747520818398232576, 747530348561571840]","[117754884, 154605692]",['No airlines involved'],0.5,"[1, 0]",['@easyJet how nice of you not to reply to my ...,"['Mon Jun 27 20:03:46 +0000 2016', 'Mon Jun 27..."


## User sentiment before and after customer support

In [5]:
#helper functions
def append_row(df, entry):
    assert len(df.columns) == len(entry), "Not enough data"
    df2 = pd.DataFrame([entry,], columns=df.columns)
    return df.append(df2, ignore_index=True)

def time_delta(datestring1, datestring2):
    date1 = datetime.datetime.strptime(datestring1, '%a %b %d %H:%M:%S +0000 %Y')
    date2 = datetime.datetime.strptime(datestring2, '%a %b %d %H:%M:%S +0000 %Y')
    timedelta = date2 - date1
    return(timedelta.seconds)

In [74]:
def add_sentiment_change(df):
    total = 0
    errors = 0
    
    airline_names = ["KLM", "AirFrance", "British_Airways", "AmericanAir", "Lufthansa", 
                    "AirBerlin", "AirBerlin assist", "easyJet", "RyanAir", "SingaporeAir", 
                    "Qantas", "EtihadAirways", "VirginAtlantic"]
    airline_ids = ["56377143", "106062176", "18332190", "22536055", 
                    "124476322", "26223583", "2182373406", "38676903", 
                    "1542862735", "253340062", "218730857", "45621423", "20626359"]
    
    
    df_interaction = pd.DataFrame(columns=['replyTime', 'sentChange'])
    sentchange = []
    #find interactions
    for data in df[['user_ids', 'sentiments', 'dates','airlines_involved', 'conversation_length']].values:
        total +=1
        if (total % 1000 == 0):
            print("at" + str(total))
            
        all_ids = eval(data[0])
        all_airlinesinvolved = eval(data[3])
        current_sentchanges = []
        NoError = False
        
        if (data[3] == "['No airlines involved]") or (data[4] < 3):
            sentchange.append([])
            #error += 1
        else:
            try: 
                all_sentiments = eval(data[1])
                all_dates = eval(data[2])
            
                sentiment_before = []
                sentiment_after = []
                StateIsAfter = False
                NoError = True
            except NameError:
                errors += 1
                
            if NoError:
                i = 0
                while i < len(all_ids):
                    while (i < len(all_ids)) and (str(all_ids[i]) not in airline_ids):
                        if StateIsAfter:
                            sentiment_after.append(all_sentiments[i])
                        else:
                            sentiment_before.append(all_sentiments[i])
                        i += 1

                    if StateIsAfter:
                        if ((len(sentiment_before) > 0) and (len(sentiment_after) > 0)):
                            sentimpact = sum(sentiment_before)/len(sentiment_before) - sum(sentiment_after)/len(sentiment_after)
                            current_sentchanges.append(sentimpact)
                            StateIsAfter = False
                            sentiment_before = sentiment_after
                            sentiment_after = []
                            i += 1
                        else:
                            StateIsAfter = False
                            i += 1
                    else:
                        StateIsAfter = True
                        i += 1

    #             except NameError:
    #                 errors += 1
                #print(current_sentchanges)
                sentchange.append(current_sentchanges)
            else:
                sentchange.append([])
        
    
    df["SentChange"] = sentchange
    
    print('errors: ' + str(errors))
    print('total: ' + str(total))
    return df

In [75]:
df = add_sentiment_change(df)

at1000
at2000
at3000
at4000
at5000
at6000
at7000
at8000
at9000
at10000
at11000
at12000
at13000
at14000
at15000
at16000
at17000
at18000
at19000
at20000
at21000
at22000
at23000
at24000
at25000
at26000
at27000
at28000
at29000
at30000
at31000
at32000
at33000
at34000
at35000
at36000
at37000
at38000
at39000
at40000
at41000
at42000
at43000
at44000
at45000
at46000
at47000
at48000
at49000
at50000
at51000
at52000
at53000
at54000
at55000
at56000
at57000
at58000
at59000
at60000
at61000
at62000
at63000
at64000
at65000
at66000
at67000
at68000
at69000
at70000
at71000
at72000
at73000
at74000
at75000
at76000
at77000
at78000
at79000
at80000
at81000
at82000
at83000
at84000
at85000
at86000
at87000
at88000
at89000
at90000
at91000
at92000
at93000
at94000
at95000
at96000
at97000
at98000
at99000
at100000
at101000
at102000
at103000
at104000
at105000
at106000
at107000
at108000
at109000
at110000
at111000
at112000
at113000
at114000
at115000
at116000
at117000
at118000
at119000
at120000
at121000
at122000
at123000
a

In [88]:
#df['SentChange'].info()
df[df.astype(str)['SentChange'] != '[]']

,Unnamed: 0,False,conversation_id,raw_tweets_info,conversation_length,tweet_ids,user_ids,airlines_involved,mean_sentiment,sentiments,full_text,dates,SentChange
6,6,6,29,"[(747529324358737921, 43028454), (747526420449...",3,"[747470611002331136, 747526420449075200, 74752...","[43028454, 18332190, 43028454]",['British_Airways'],0.000000,"[0, 0, 0]",['@British_Airways I am thinking of booking a ...,"['Mon Jun 27 16:44:15 +0000 2016', 'Mon Jun 27...",[0.0]
52,52,52,238,"[(747518975861481473, 4897552792), (7474571214...",3,"[746060507132551168, 747457121487314944, 74751...","[4897552792, 38676903, 4897552792]",['easyJet'],1.333333,"[2, 2, 0]","[""@KLM Is the IPhone App working, all of a sud...","['Thu Jun 23 19:21:00 +0000 2016', 'Mon Jun 27...",[2.0]
151,151,151,725,"[(747489335960797184, 2792150643), (7474870309...",4,"[746028652589113344, 746057572654981120, 74748...","[2792150643, 2792150643, 38676903, 2792150643]",['easyJet'],0.250000,"[0, 1, 0, 0]",['@StephenCur Yes if you have confirmation fro...,"['Thu Jun 23 17:14:25 +0000 2016', 'Thu Jun 23...",[0.5]
167,167,167,790,"[(747486121010036737, 4830697389), (7474837636...",3,"[747418252771483648, 747483763647610880, 74748...","[225883022, 38676903, 4830697389]",['easyJet'],0.000000,"[0, 0, 0]","[""se vcs manda um 'podemos prosseguir?' e eu r...","['Mon Jun 27 13:16:12 +0000 2016', 'Mon Jun 27...",[0.0]
189,189,189,884,"[(747479155214815232, 44423326), (747425502277...",3,"[745876909158993920, 747425502277812224, 74747...","[44423326, 38676903, 44423326]",['easyJet'],0.000000,"[0, 0, 0]",['@Ryanair can I take a suit bag and little sp...,"['Thu Jun 23 07:11:27 +0000 2016', 'Mon Jun 27...",[0.0]
259,259,259,1149,"[(747463323701092352, 241112930), (74746236141...",3,"[747454797050175488, 747462361414504448, 74746...","[241112930, 2182373406, 241112930]",['AirBerlin assist'],0.000000,"[0, 0, 0]","[""@AmericanAir wonder what kind of treat I'm i...","['Mon Jun 27 15:41:25 +0000 2016', 'Mon Jun 27...",[0.0]
289,289,289,1300,"[(747453981836840961, 29170825), (747453666693...",4,"[747440864495083520, 747450530675437568, 74745...","[14582769, 1542862735, 29170825, 29170825]",['RyanAir'],0.750000,"[1, 1, 0, 1]","[""@AmericanAir longer delay no crew. It's cle...","['Mon Jun 27 14:46:03 +0000 2016', 'Mon Jun 27...",[0.5]
291,291,291,1308,"[(747453685454770177, 57650131), (746634218818...",4,"[745214469358428160, 746610630786973696, 74663...","[57650131, 38676903, 57650131, 57650131]",['easyJet'],1.000000,"[1, 1, 1, 1]","[""Can't explain my anger at being misinformed ...","['Tue Jun 21 11:19:09 +0000 2016', 'Sat Jun 25...",[0.0]
334,334,334,1561,"[(747438210754744320, 412511466), (74743253351...",3,"[745772750166499328, 747432533516816384, 74743...","[412511466, 38676903, 412511466]",['easyJet'],1.000000,"[0, 1, 2]",['Am I allowed to take my DJI Phantom 3 onto a...,"['Thu Jun 23 00:17:34 +0000 2016', 'Mon Jun 27...",[-2.0]
345,345,345,1622,"[(747434096524148736, 713489638330122240), (74...",3,"[747415842791489536, 747433835948810240, 74743...","[713489638330122240, 56377143, 713489638330122...",['KLM'],1.000000,"[1, 0, 2]","['Got live person, asks ridiculous question an...","['Mon Jun 27 13:06:37 +0000 2016', 'Mon Jun 27...",[-1.0]


In [86]:
df.to_csv('conversations_with_sentimpact.csv')

In [87]:
df

,Unnamed: 0,False,conversation_id,raw_tweets_info,conversation_length,tweet_ids,user_ids,airlines_involved,mean_sentiment,sentiments,full_text,dates,SentChange
0,0,0,5,"[(747530773037780992, 31132554), (747526794446...",2,"[747526794446802944, 747530773037780992]","[22873368, 31132554]",['No airlines involved'],NaN,"[nan, 1]",['Hey @British_Airways our @easyJet flight Ven...,"['Mon Jun 27 20:27:30 +0000 2016', 'Mon Jun 27...",[]
1,1,1,6,"[(747530681509625860, 417945325), (74745123859...",2,"[747451238598475776, 747530681509625860]","[55904765, 417945325]",['No airlines involved'],0.000000,"[0, 0]",['@easyJet thanks for looking after my luggage...,"['Mon Jun 27 15:27:16 +0000 2016', 'Mon Jun 27...",[]
2,2,2,11,"[(747530531475173376, 597673831), (74752831114...",2,"[747528311144734720, 747530531475173376]","[427951650, 597673831]",['No airlines involved'],0.500000,"[1, 0]","[""@British_Airways I booked via Expedia. I'm n...","['Mon Jun 27 20:33:32 +0000 2016', 'Mon Jun 27...",[]
3,3,3,12,"[(747530509098450944, 1624311463), (7475297117...",2,"[747529711782244352, 747530509098450944]","[1624311463, 1624311463]",['No airlines involved'],0.000000,"[0, 0]",['@easyJet is there anyway to travel with 2 ca...,"['Mon Jun 27 20:39:06 +0000 2016', 'Mon Jun 27...",[]
4,4,4,14,"[(747530348561571840, 154605692), (74752081839...",2,"[747520818398232576, 747530348561571840]","[117754884, 154605692]",['No airlines involved'],0.500000,"[1, 0]",['@easyJet how nice of you not to reply to my ...,"['Mon Jun 27 20:03:46 +0000 2016', 'Mon Jun 27...",[]
5,5,5,17,"[(747530128947740672, 3929595672), (7474305579...",2,"[747430557924024320, 747530128947740672]","[3838021994, 3929595672]",['No airlines involved'],1.000000,"[1, 1]","['@amyrose2392 Hey, sorry for the late reply. ...","['Mon Jun 27 14:05:06 +0000 2016', 'Mon Jun 27...",[]
6,6,6,29,"[(747529324358737921, 43028454), (747526420449...",3,"[747470611002331136, 747526420449075200, 74752...","[43028454, 18332190, 43028454]",['British_Airways'],0.000000,"[0, 0, 0]",['@British_Airways I am thinking of booking a ...,"['Mon Jun 27 16:44:15 +0000 2016', 'Mon Jun 27...",[0.0]
7,7,7,53,"[(747528315452416000, 1032224154), (7474209962...",2,"[747420996232163328, 747528315452416000]","[1032224154, 1032224154]",['No airlines involved'],NaN,"[1, nan]",['.@easyJet both flights been delayed by 2+ ho...,"['Mon Jun 27 13:27:06 +0000 2016', 'Mon Jun 27...",[]
8,8,8,56,"[(747528297282699264, 3860009236), (7474514991...",2,"[747451499102539776, 747528297282699264]","[259582885, 3860009236]",['No airlines involved'],0.000000,"[0, 0]",['@easyJet trying to add seats to booking made...,"['Mon Jun 27 15:28:19 +0000 2016', 'Mon Jun 27...",[]
9,9,9,58,"[(747528180848680964, 23534495), (747465580945...",3,"[747448843353001984, 747465580945412096, 74752...","[23534495, 627363, 23534495]",['No airlines involved'],0.000000,"[0, 0, 0]",['@easyJet do i believe the app that says we a...,"['Mon Jun 27 15:17:45 +0000 2016', 'Mon Jun 27...",[]
